# MP 1: Introduction to Klampt, Coordinates, and Transforms

**Due date**: Feb 4, 2021 at 10:45am.

**Instructions**: Read and complete the problems below. To submit your assignment, perform the following:

1. Double-check that your notebook runs without error from a clean "Restart and Clear Output".
2. Download your completed notebook.  To avoid sending unnecessarily large notebooks, choose "Restart and Clear Output" before downloading. 
3. Submit your notebook (and any other new files you created) on Moodle [http:/learn.illinois.edu](http:/learn.illinois.edu).


In [1]:
#Imports

#If you have wurlitzer installed, this will help you catch printouts from Klamp't
#Note: doesn't work on Windows
#%load_ext wurlitzer

import time
from klampt import *
from klampt import vis
from klampt.math import vectorops,so3,se3
from klampt.model.trajectory import Trajectory
import math
import random
vis.init('IPython')
pass

## Coordinates Warm-Up

The cells below display two endpoints of a line segment, which move according to the functions `source_motion(t)` and `target_motion(t)`. The on-screen display incorrectly calculates the polar coordinates of this segment, printing out 0,0,0. Your job is to properly calculate these values.

Complete the function `problem1_polar(point1,point2)` which should return a tuple `(distance,elevation,azimuth)` describing the polar coordiantes of the vector from point1 to point2.

Hint: run the `selfTest()` function to verify that you are performing the calculations properly.


In [2]:
########################## Your code goes in this cell ################################

def problem1_polar(point1,point2):
    #TODO: Return a tuple (distance,elevation,azimuth), where distance
    #is the length of the segment with endpoints point1 and point2,
    #elevation is the angle between the equator and the displacement vector,
    #and azimuth is its heading along the x-y plane.
    #Both elevation and azimuth should be in degrees, with elevation in
    #the range [-90,90] and azimuth in the range [0,360).
    #    
    #As usual, azimuth 0 points along the +x axis, 90 points along +y,
    #180 points along -x, and 270 points along -y
    displacement = vectorops.sub(point2,point1)
    distance = (displacement[0]**2 + displacement[1]**2 + displacement[2]**2)**0.5
    
    if displacement[0] != 0 and displacement[1] != 0:
        elevation = math.degrees(math.atan(displacement[2] / (displacement[0]**2 + displacement[1]**2)**0.5))
    elif displacement[2] == 0:
        elevation = 0
    else:
        elevation = 90 if displacement[2] > 0 else -90
    
    if displacement[0] > 0:
        azimuth = math.degrees(math.atan(displacement[1] / displacement[0]))
        azimuth += 360 if azimuth < 0 else 0
    elif displacement[0] < 0:
        azimuth = math.degrees(math.atan(displacement[1] / displacement[0])) + 180
    elif displacement[1] == 0:
        azimuth = 0
    else:
        azimuth = 90 if displacement[1] > 0 else 270
    
    return (distance,elevation,azimuth)

def source_motion(t):
#     return 0,0,0
    #comment out the above line to make sure your lab1a routine works when the source is modified
    return math.sin(t),math.cos(t),0

def target_motion(t):
    return 1+math.sin(t+math.pi),math.cos(t*2),math.sin(t*0.1+0.5)

In [3]:
#testing... run this cell to verify your code

def fuzzy_eq(a,b,eps=1e-8):
    """Returns true if a is within +/- eps of b."""
    return abs(a-b)<=eps

def fuzzy_veq(a,b,eps=1e-8):
    """Returns true if a is within +/- eps of b."""
    if len(a) != len(b): return False
    return all(fuzzy_eq(i,j) for (i,j) in zip(a,b))

def selfTest():
    """You may use this function to make sure your values are correct"""
    assert fuzzy_veq(problem1_polar((0,0,0),(1,0,0)), (1,0,0))
    assert fuzzy_veq(problem1_polar((0,0,0),(10,0,0)), (10,0,0))
    assert fuzzy_veq(problem1_polar((0,0,0),(0,10,0)), (10,0,90))
    assert fuzzy_veq(problem1_polar((0,0,0),(0,-10,0)), (10,0,270))
    assert fuzzy_veq(problem1_polar((55,55,0),(55,45,0)), (10,0,270))
    assert fuzzy_veq(problem1_polar((55,55,0),(55,55,0)), (0,0,0))
    #test whether length is symmetric, and azimuth has a symmetry +/- 180
    val1 = problem1_polar((-53,74,34),(39.3,93.5,12.3))
    val2 = problem1_polar((39.3,93.5,12.3),(-53,74,34))
    assert val1[0] == val2[0]
    assert fuzzy_eq(180+val1[2],val2[2])
    return
selfTest()

In [4]:
#run this cell to show a visualization playback window calling your code
pos1 = (0,0)
pos2 = (2,0)
t = 0

def boilerplate_start():
    global pos1,pos2,t
    vis.add("source", (0,0,0), size=0.05)
    vis.setColor("source",1,0,0)
    vis.add("target", (2,0,0), size=0.05)
    vis.setColor("target",0,0,1)
    vis.add("line", Trajectory([0,1],[[0,0,0],[2,0,0]]))
    vis.setColor("line",1,1,0)
    #kvis.addText("HUD1",2,2)
    vis.addText("HUD1",text="",position = (2,2))
    pos1 = (0,0)
    pos2 = (2,0)
    t = 0

def boilerplate_advance():
    global pos1,pos2,t
    t += 0.02
    pos1 = source_motion(t)
    pos2 = target_motion(t)
    vis.add("source",(pos1[0],pos1[1],0))
    vis.add("target",(pos2[0],pos2[1],0))
    vis.add("line", Trajectory([0,1],[[pos1[0],pos1[1],0],[pos2[0],pos2[1],0]]))
    #draw text
    (length,elevation,azimuth) = problem1_polar(pos1,pos2)
    vis.addText("HUD1",text="Length %f, elevation %f, azimuth %f"%(length,elevation,azimuth))
    return

from IPython.display import display,clear_output
import ipywidgets as widgets
from klampt.vis.ipython import Playback

vis.createWindow()

playback_widget = Playback(vis.nativeWindow())
playback_widget.quiet = False
playback_widget.reset = boilerplate_start
playback_widget.advance = boilerplate_advance

boilerplate_start()
vis.show()
display(playback_widget)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

KlamptWidgetAdaptor(rpc={'type': 'multiple', 'calls': [{'type': 'add_sphere', 'name': 'source', 'x': 0, 'y': 0…

Playback(children=(HBox(children=(Button(description='Play', icon='play', style=ButtonStyle(), tooltip='Start …

## Transformations

The cell below displays a car-like vehicle. By default the car will use automatic controls (chosen at random by default), but you can also drive it around using the arrow buttons.  The display of the car is broken!  The wheels are sideways, it does not properly rotate, and the front wheels do not properly indicate the steering angle.  It is your job to fix it.

A. The wheels are incorrectly oriented upon loading, with the cylinders pointing in the Z direction rather than the Y direction.  Implement a 3D rotation matrix `R` in the indicated line that will properly orient the wheels.

B. Implement `problem2_config_to_xform(config)`. Here an `config` is a tuple `(tx,ty,angle)` indicating the translation and rotation of a 2D rigid body transformation.  The output will be a 3D transform.  When this is implemented, the car body should rotate as the car turns.

C. Complete `problem2_compose_config_xforms(config1,config2)`. This function composes two transforms together so that applying the result to a point is equivalent to first applying the transform of config2, and then applying the transform of config1. When this is correctly implemented, the front wheels should rotate to indicate steering angle.

You may wish to study the [Klampt math modules](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/Manual-Math.html). In particular, the so3 and se3 modules will be useful here.  Also, to help you debug, you can change the automatic controls by editing the `problem2_control()` function.  Return values `up` and `down` increase and decrease the velocity, and `left` and `right` increase and decrease the steering angle.


In [25]:
########################## Your code goes in this cell ################################

def problem2_config_to_xform(config):
    """TODO: return the Klamp't se3 element corresponding to the given SE(2)
    configuration. 
    
    In:
    - config: a tuple (tx,ty,angle) indicating the translation amount (tx,ty)
      and the rotation angle about the Z axis (in degrees)
      
    Out:
    - pair (R,t): a Klamp't se3 element giving the 3D transform.
      R is 9 element list, listing the entries of the rotation matrix in
      column-major form. t is the 3D translation.
    """
    #TODO: fill in your code here
    return so3.from_rpy((0,0,config[2])),[config[0],config[1],0]

def problem2_apply_config_xform(config,point):
    """Apply the 3D rigid transform corresponding to the SE(2) configuration
    to a 3D point P.
    
    In:
    - config: a tuple (tx,ty,angle) indicating the translation amount (tx,ty)
      and the rotation amount angle (in degrees)
    - point: a pair (px,py,pz) indicating the coordinates of P in the original
      frame.
      
    Out:
    - a triple (qx,qy,qz) indicating the coordinates of P after rotating by
      angle degrees and then translating by (tx,ty). 
    """
    return se3.apply(problem2_config_to_xform(config), point)

def problem2_compose_config_xforms(config1,config2):
    """TODO: compose the transforms according to the SE(2) transforms. 
    In other words, performs the operation xform1*xform2, where
    xform1 is the SE(3) transform corresponding to config1 and xform2
    is the SE(3) transform corresponding to config2.
    
    In:
     - config1: a tuple (tx1,ty1,angle1) (angles in degrees)
     - config2: a tuple (tx2,ty2,angle2) (angles in degrees)
    Out:
     - xform: an SE(3) representing the composed transforms. In other words,
       for a 3D point p, application of xform to p is equivalent to applying
       the transform of config2, followed by applying the transform of config1.
     """
    xform1 = problem2_config_to_xform(config1)
    xform2 = problem2_config_to_xform(config2)
    return se3.mul(xform1, xform2)


#body
car_body = None
#fl, fr, bl, br
wheels = {'fl':None,'fr':None,'bl':None,'br':None}

world = WorldModel()
car_body = world.loadElement("data/body.obj")
wheels['fl'] = world.loadElement("data/tire_fl.obj")
wheels['fr'] = world.loadElement("data/tire_fr.obj")
wheels['bl'] = world.loadElement("data/tire_bl.obj")
wheels['br'] = world.loadElement("data/tire_br.obj")
car_body = world.rigidObject(car_body)
car_body.appearance().setColor(1,0,0,1)
for (k,v) in wheels.items():
    wheels[k] = world.rigidObject(v)
    #TODO: fix the wheel geometry by applying a rotation here
    R = so3.from_rpy((math.pi / 2, 0, 0))
    wheels[k].geometry().transform(R,[0,0,0])
    wheels[k].appearance().setColor(0.3,0.3,0.3,1)

def problem2_control():
    return random.choice(['left','right','up','down'])


In [26]:
#run this cell to bind your code to the visualization playback window

from klampt import vis
import ipywidgets as widgets
from klampt.vis.ipython import Playback

velocity = 0
steer = 0
vinc = 0.04
vmin = -1
vmax = 1
steermax = 45
steerinc = 2
config = [0,0,0]
random_control = True

def boilerplate_start():
    global world,config,velocity,steer
    #restore the world state
    velocity = 0
    config = [0,0,0]
    steer = 0
    update_car()
    vis.update()

def update_car():
    global config,steer
    steeringAngle = math.radians(steer)
    frontTirePos1 = (0.50,0.16,0)
    frontTirePos2 = (0.50,-0.16,0)
    rearTirePos1 = (0,0.16,0)
    rearTirePos2 = (0,-0.16,0)
    rearPoint1 = problem2_apply_config_xform(config,rearTirePos1)
    rearPoint2 = problem2_apply_config_xform(config,rearTirePos2)
    xform = problem2_config_to_xform(config)
    R,t = car_body.getTransform()
    body_height = t[2]
    car_body.setTransform(xform[0],[xform[1][0],xform[1][1],body_height])
    wheels['bl'].setTransform(xform[0],rearPoint1)
    wheels['br'].setTransform(xform[0],rearPoint2)
    tire1xform = problem2_compose_config_xforms(config,(frontTirePos1[0],frontTirePos1[1],steer))
    tire2xform = problem2_compose_config_xforms(config,(frontTirePos2[0],frontTirePos2[1],steer))
    wheels['fl'].setTransform(*tire1xform)
    wheels['fr'].setTransform(*tire2xform)
    return

def update_config(dt=0.02):
    global config,velocity,steer
    rangle = math.radians(config[2])
    v = [velocity*math.cos(rangle),velocity*math.sin(rangle)]
    dtheta = velocity*steer*5
    config = (config[0]+v[0]*dt,config[1]+v[1]*dt,config[2]+dtheta*dt)

def boilerplate_advance():
    global random_control
    update_config()
    update_car()
    if random_control:
        if drive_button is None:
            u = problem2_control()
            #print xform,u
            boilerplate_keypress(u)
        else:
            boilerplate_keypress(drive_button)
    #boilerplate_keypress(random.choice(['up','down','left','right']))

def boilerplate_keypress(c):
    global steer,steerinc,velocity,vinc,random_control
    if c.startswith('Arrow'):
        random_control = False
        boilerplate_keypress(c[5:].lower())
    elif c=='w':
        random_control = False
        boilerplate_keypress('up')
    elif c=='a':
        random_control = False
        boilerplate_keypress('left')
    elif c=='s':
        random_control = False
        boilerplate_keypress('down')
    elif c=='d':
        random_control = False
        boilerplate_keypress('right')
    elif c=='up':
        velocity += vinc
        if velocity > vmax:
            velocity = vmax
    elif c=='down':
        velocity -= vinc
        if velocity < vmin:
            velocity = vmin
    elif c=='right':
        steer -= steerinc
        if steer < -steermax:
            steer = -steermax
    elif c=='left':
        steer += steerinc
        if steer > steermax:
            steer = steermax
    elif c=='r':
        random_control = True

drive_button = None
layout = widgets.Layout(width='30px', height='30px')
left = widgets.Button(description='',icon='toggle-left', layout=layout)
right = widgets.Button(description='',icon='toggle-right', layout=layout)
up = widgets.Button(description='',icon='toggle-up', layout=layout)
down = widgets.Button(description='',icon='toggle-down', layout=layout)
stop = widgets.Button(description='',icon='hand-stop-o', layout=layout)
def driveset(b):
    global drive_button
    if b is left: drive_button = 'left'
    elif b is right: drive_button = 'right'
    elif b is up: drive_button = 'up'
    elif b is down: drive_button = 'down'
    elif b is stop: drive_button = None
    else: print("Invalid drive button",b)
left.on_click(driveset)
right.on_click(driveset)
up.on_click(driveset)
down.on_click(driveset)
stop.on_click(driveset)

vis.createWindow()
vis.add("world",world)

playback_widget = Playback(vis.nativeWindow())
playback_widget.quiet = False
playback_widget.reset = boilerplate_start
playback_widget.advance = boilerplate_advance

boilerplate_start()

vis.show()
display(playback_widget)
display(widgets.HBox([left,right,down,up,stop]))

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

KlamptWidgetAdaptor(scene={'metadata': {'type': 'Object', 'version': 4.4, 'fullscene': True, 'generator': 'Kla…

Playback(children=(HBox(children=(Button(description='Play', icon='play', style=ButtonStyle(), tooltip='Start …

## Robot Transformations

A robot and an object are drawn below. In this problem, you will calculate various coordinate transforms.  The two-armed robot TRINA is posed in front of a table, with an object on it.  The coordinate frames of the object, each of the grippers, and an overhead camera are displayed.

A. Implement `problem3_object_center_local` to calculate the local coordinates of the object's center with respect to a gripper's frame.

B. Implement `problem3_object_point_local` to calculate the local coordinates of a point on the object, with respect to a gripper's frame.  Also, implement `problem3_object_direction_local` to do the same, but for a directional quantity.

C. Implement `problem3_camera_point_local` to calculate the local coordinates of a point on an object with respect to a gripper's frame, but only when the object's pose is known relative to the camera frame.

Test your code in the cell below.  Make sure that the quantities make sense visually; to view a point in the visualization, try `vis.add("point",se3.apply(X_gripper.getTransform(),local_pt))` where X is either left or right.  Moreover, make sure that your quantities make sense when you translate and rotate the object.

In [45]:
#Display the robot and object

from klampt.vis.ipython import EditPoint,EditTransform
from IPython.display import display,Markdown

world = WorldModel()
world.readFile("../data/robots/TRINA.urdf")
world.readFile("../data/terrains/plane.env")
obj = world.makeRigidObject("object")
obj.geometry().loadFile("../data/objects/ycb-select/011_banana/nontextured.ply")
obj.setTransform(so3.identity(),[1,0,0.75])
obj.appearance().setColor(1,1,0)
robot = world.robot(0)

from klampt.model.create import primitives
table = primitives.box(0.5,0.5,0.04,center=(1,0,0.73),world=world,name='table',mass=float('inf'))
table.appearance().setColor(0.4,0.3,0.2)

from klampt.io import resource
resource.setDirectory("../data/resources/TRINA")
qhome = resource.get("home.config")
robot.setConfig(qhome)

left_gripper = robot.link("left_gripper:base_link")
right_gripper = robot.link("right_gripper:base_link")

camera_xform = (so3.mul(so3.rotation((0,1,0),2.7),so3.rotation((0,0,1),-math.pi/2)),[0.5,0,1.9])

def edit_object_callback(xform):
    obj.setTransform(*xform)
    vis.update()

vis.createWindow()

edit_object = EditTransform(obj.getTransform(),[0.75,-0.25,0.75],[1.25,0.25,0.76],
                            klampt_widget=vis.nativeWindow(),callback=edit_object_callback)

vis.add("camera_xform",camera_xform)
vis.add("left_gripper_xform",left_gripper.getTransform())
vis.add("right_gripper_xform",right_gripper.getTransform())


vis.add("world",world)
vis.show()
display(Markdown("### Object xform"))
display(edit_object)

KlamptWidgetAdaptor(rpc={'type': 'multiple', 'calls': [{'type': 'add_xform', 'name': 'edited_xform', 'length':…

### Object xform

In [43]:
########################## Your code goes in this cell ################################

def problem3_object_center_local(obj,gripper_link):
    """Return the 3D coordinates of the object's origin relative to
    the gripper link's frame.
    
    In:
        obj: RigidObjectModel instance.
        gripper_link: RobotModelLink instance.
    
    Out:
        3-vector giving the coordinates of obj's origin, relative to
        the reference frame of gripper_link.
    """
    obj_T = obj.getTransform()
    link_T = gripper_link.getTransform()
#     print(obj_T)
#     print(link_T)
    
    return vectorops.sub(link_T[1], obj_T[1])

def problem3_object_point_local(point_obj,obj,gripper_link):
    """Return the 3D coordinates of point_obj, expressed in
    the object's frame, relative to the gripper link's frame.
    
    In:
        point_obj: 3-vector giving the coordinates of a point P
            relative to obj.
        obj: RigidObjectModel instance.
        gripper_link: RobotModelLink instance.
    
    Out:
        3-vector giving the coordinates of P, relative to the
        reference frame of gripper_link.
    """
    t_obj = problem3_object_center_local(obj, gripper_link)
    
    return vectorops.add(t_obj,point_obj)

def problem3_object_direction_local(dir_obj,obj,gripper_link):
    """Return the 3D coordinates of direction dir_obj, expressed in
    the object's frame, relative to the gripper link's frame.
    
    In:
        dir_obj: 3-vector giving the coordinates of a direction D
            relative to obj.
        obj: RigidObjectModel instance.
        gripper_link: RobotModelLink instance.
    
    Out:
        3-vector giving the coordinates of D, relative to the
        reference frame of gripper_link.
    """
    t_obj = problem3_object_center_local(obj, gripper_link)
    point = vectorops.add(t_obj, dir_obj)
    
    return vectorops.unit(point)

def problem3_camera_point_local(point_obj,xform_camera,xform_obj_camera,gripper_link):
    """Converts a point on the object to coordinates in the
    gripper link's frame.  Only the object's transform relative to the
    camera are known.
    
    In:
        point_obj: 3-vector expressing a point P relative to an object.
        xform_camera: the coordinates of the camera, in the world frame.
        xform_obj_camera: the coordinates of the object, in the camera frame
            (e.g., produced by an object pose estimator)
        gripper_link: RobotModelLink instance.
    """
    return point_obj


In [44]:
print("A. Coordinates of object w.r.t. left gripper:",problem3_object_center_local(obj,left_gripper))
print("   Coordinates of object w.r.t. right gripper:",problem3_object_center_local(obj,right_gripper))
stem = (0.05,0.08,0.01)
tip = (-0.02,-0.08,0.01)
axis = vectorops.unit(vectorops.sub(stem,tip))
#vis.add("stem",se3.apply(obj.getTransform(),stem),color=(0,1,0),size=0.02)
#vis.add("tip",se3.apply(obj.getTransform(),tip),color=(0,0,1),size=0.02)
obj_pose_estimate = se3.mul(se3.inv(camera_xform),obj.getTransform())
print("B. Coordinates of stem w.r.t. left gripper:",problem3_object_point_local(stem,obj,left_gripper))
print("   Coordinates of stem w.r.t. right gripper:",problem3_object_point_local(stem,obj,right_gripper))
print("   Coordinates of tip w.r.t. left gripper:",problem3_object_point_local(tip,obj,left_gripper))
print("   Coordinates of tip w.r.t. right gripper:",problem3_object_point_local(tip,obj,right_gripper))
print("   Coordinates of axis w.r.t. left gripper:",problem3_object_direction_local(axis,obj,left_gripper))
print("   Coordinates of axis w.r.t. right gripper:",problem3_object_direction_local(axis,obj,right_gripper))
print("C. Coordinates of stem w.r.t. left gripper:",problem3_camera_point_local(stem,camera_xform,obj_pose_estimate,left_gripper))
print("   Coordinates of stem w.r.t. right gripper:",problem3_camera_point_local(stem,camera_xform,obj_pose_estimate,right_gripper))

A. Coordinates of object w.r.t. left gripper: [-0.27010082316313944, 0.1581581044056813, 0.39434302015792855]
   Coordinates of object w.r.t. right gripper: [-0.2942909908780641, -0.19811005013402638, 0.389858737546805]
B. Coordinates of stem w.r.t. left gripper: [-0.22010082316313945, 0.2381581044056813, 0.40434302015792856]
   Coordinates of stem w.r.t. right gripper: [-0.2442909908780641, -0.11811005013402638, 0.399858737546805]
   Coordinates of tip w.r.t. left gripper: [-0.29010082316313945, 0.07815810440568129, 0.40434302015792856]
   Coordinates of tip w.r.t. right gripper: [-0.3142909908780641, -0.2781100501340264, 0.399858737546805]
   Coordinates of axis w.r.t. left gripper: [0.11348574884825571, 0.9326908459682259, 0.342357664811854]
   Coordinates of axis w.r.t. right gripper: [0.1292857502443073, 0.8714461795561264, 0.4731455916742732]
C. Coordinates of stem w.r.t. left gripper: (0.05, 0.08, 0.01)
   Coordinates of stem w.r.t. right gripper: (0.05, 0.08, 0.01)
